In [1]:
%display latex
import sage.misc.banner; sage.misc.banner.banner()

┌────────────────────────────────────────────────────────────────────┐
│ SageMath version 10.7, Release Date: 2025-08-09                    │
│ Using Python 3.12.11. Type "help()" for help.                      │
└────────────────────────────────────────────────────────────────────┘


## Second example: dimension of $\bf{U}_A[p]$ from the Kraft word of $A[p]$ 

Given a Kraft word $w$, return the dimension of $U_A[p]$ where $A[p]$ is the $BT_1$ corresponding to $w$.

In [2]:
g = 4
p = 3
q = p^2
K = GF(q)
n = 1

In [3]:
R = PolynomialRing(K,'x',n*g*(2*g-1))

# source code in: sage/rings/padics/witt_vector_ring.py
WR = WittVectorRing(R, p=p, prec=n, algorithm='finotti')
WR0,WR1,WRp = WR.zero(),WR.one(),p*WR.one()

def wittFrob(w,W):
    t = list(w.coordinates())
    for i in range(len(t)):
        t[i]=t[i]^p
    return W(t)

In [4]:
# define the A matrix
A = matrix(WR,2*g,2*g)
idx = 0
for i in range(2*g):
    for j in range(i):
        A[i,j] = WR(R.gens()[n*idx:n*(idx+1)])
        A[j,i] = -A[i,j]
        idx += 1
del idx
Apt = A.apply_map(lambda x:wittFrob(x,WR)).transpose()

In [5]:
# convert a square sparse column array into a matrix
def CSCtoMat(ring,csc):
    m = matrix(ring,len(csc),len(csc))
    for i in range(len(csc)):
        for j,v in csc[i]:
            m[j,i]=v
    return m

# multiply a matrix (left) with a sparse column array (right)
def mul(mat,csc):
    Nr,Nc = mat.nrows(),len(csc)
    retval = matrix(mat.base_ring(),Nr,Nc)
    for i in range(Nr):
        for j in range(Nc):
            for k,v in csc[j]:
                retval[i,j] += v*(mat[i,k])
    return retval

# direct sum of square sparse column arrays
def dirSumCSC(csc1,csc2):
    csc = csc1[:]
    for col in csc2:
        csc.append([(el[0]+len(csc1),el[1]) for el in col])
    return csc

In [6]:
# calculate the dimension given F and DF transpose
def dimension(g,n,F,DFt):
    S = (mul(Apt,DFt)).transpose()-mul(A,F)
    gens = [S[i][j].coordinates()[k] for i in range(2*g) for j in range(2*g) for k in range(n)]
    return ideal(gens).dimension()

The imput is a Kraft word $w$ given as a string (or as a list of strings, if the word is decomposable). The output is the sought after dimension. 

In [7]:
def indecomposableF(w):
    F,DFt = [],[]
    d = (len(w))    
    for i in range(d):
        if w[i]=='F':
            F += [[((i-1)%d,WR1)]]
        else:
            F += [[((i-1)%d,WRp)]]
    for i in range(d):
        if w[(-i)%d]=='F':
            DFt = [[((-i)%d,WRp)]] + DFt
        else:
            DFt = [[((-i)%d,WR1)]] + DFt
    return F,DFt

def Ffromw(l):
    F,DFt = [],[]
    for i in l:
        s = indecomposableF(i)
        F = dirSumCSC(F,s[0])
        DFt = dirSumCSC(DFt,s[1])
    return F,DFt

w1 = ['FFFVVVFV']
F,DFt = Ffromw(w1)
print(w1)
display(dimension(g,n,F,DFt))
print()

w2 = ['FFV','FFVVV']
F,DFt = Ffromw(w2)
print(w2)
display(dimension(g,n,F,DFt))
print()

w3 = ['FV','FV','FV','FV']
F,DFt = Ffromw(w3)
print(w3)
display(dimension(g,n,F,DFt))
print()

w4 = ['FFFFVVVV']
F,DFt = Ffromw(w4)
print(w4)
display(dimension(g,n,F,DFt))
print()


['FFFVVVFV']


4


['FFV', 'FFVVV']


4


['FV', 'FV', 'FV', 'FV']


6


['FFFFVVVV']


3